# State of the Climate 2025: Antarctic surface mass balance

ATL15 surface height change for 2025

In [ ]:
SOTC_YEAR = 2025

In [ ]:
import pathlib
import itertools

import numpy as np
import icesat2_smb as is2smb
import xarray as xr
import rasterio as rio
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import colormaps as mcmap
from icesat2_smb.plot_params import props_var

In [ ]:
# autoreload of modules
%load_ext autoreload
%autoreload 2

In [ ]:
# Data parent directory
dir_data = pathlib.Path('/mnt/c/Users/taryn/Data')

# ICESat-2 data (Antarctica)
# -- ATL15 Gridded Land Ice Height Change
# -- Info and latest version: https://nsidc.org/data/atl15
f_ATL15 = list(dir_data.joinpath('ATL15').glob('ATL15_A*.nc'))

# ATL15 grounded ice coverage mask
# -- Rasterized version of mask on IMBIE website, provided by Tyler Sutterley
f_mask = list(dir_data.joinpath('ATL15/ATL15_AA_10km_003_IMBIE2_v1.6').glob('*.nc'))

# ERA5 SMB anomaly
f_ERA5 = dir_data.joinpath('SOTC_2025/era5_SMB_2025_Anoms_Monthly.nc')

In [ ]:
atl15_spatial_resolution = f_ATL15[0].stem.split('_')[3]

In [ ]:
# Load data

# ATL15
ATL15_groups = [
    'delta_h', # height change relative to the ATL14 DEM [m]
    'dhdt_lag1', # quarterly height change rate [m/a]
    'dhdt_lag4', # annual height change rate [m/a]
]
atl15_original = is2smb.io.read_ATL15(
    file=f_ATL15,
    group=ATL15_groups,
    decimal_years=True,
)
atl15_original.name = "ATL15"
atl15 = atl15_original.copy()

# Grounded ice mask
# -- rename to avoid conflict with ATL15 dataset names
grounded_mask = is2smb.io.read_mask(f_mask)
grounded_mask = grounded_mask.rename({'polar_stereographic': 'Polar_Stereographic'})

# ERA5 SMB anomaly
era5_original = xr.open_dataset(f_ERA5, decode_coords=True)
era5_original = era5_original.rio.write_crs(4326)
era5_original = era5_original.assign_attrs(dict(name="ERA5"))
era5 = era5_original.copy()

# Pre-processing

## Regrid model data to match ATL15

In [ ]:
# ERA5
era5 = is2smb.spatial.interpolate_grid(
    data=era5,
    regrid_like=atl15_original.delta_h.delta_h,
)

# zero out time part of datetime to make life easier
era5['time'] = [x.astype("datetime64[D]") for x in era5.time.values]

era5

## Reduce ATL15 to only SOTC timesteps

In [ ]:
atl15['delta_h'] = atl15.delta_h.sel(time=slice(SOTC_YEAR, SOTC_YEAR+1))
atl15['dhdt_lag1'] = atl15.dhdt_lag1.sel(time=slice(SOTC_YEAR - 0.25/2, (SOTC_YEAR+1) + 0.25/2))
atl15['dhdt_lag4'] = atl15.dhdt_lag4.sel(time=slice(SOTC_YEAR - 1/2, (SOTC_YEAR+1) + 1/2))

atl15

## Apply grounded ice mask

In [ ]:
# ATL15
atl15 = atl15.map_over_datasets(
    func=is2smb.spatial.apply_mask,
    kwargs=dict(mask=grounded_mask)
)

# ERA5
era5 = is2smb.spatial.apply_mask(
    data=era5,
    mask=grounded_mask
)

## Compute rates in model

The ERA5 data that Luke shared are in $Gt\ m^{-2}$ for each month relative to the 1991-2020 climatology; multiply by $10^{12}$ to get $mm\ w.e.$ per month. I additionally multiply by $12\times10^{-3}\rho_{water}/\rho_{ice}$ to convert to $m\ a^{-1}\ i.e.$, which is what I typically work with from the GSFC-FDM.

To best compare with ICESat-2, I want the data to be analogous to the quarterly rate of height change $dhdt$ in ATL15. In the GSFC-FDM, that is the $\frac{d}{dt}$ of the cumulative SMB anomaly. 

Here we are working with _instantaneous_ SMB anomalies relative to a time-varying monthly climatology. They are already 'rates' (already have a time denominator; they are the instantaneous height change due to SMB) In this case, the rate of height change is the difference between the total SMB (anomaly + mean) at two timesteps:

$\frac{dh}{dt}$ due to SMB $= (SMB_A + \overline{SMB})(t) - (SMB_A + \overline{SMB})(t-\Delta t)$

note, we do _not_ divide by $\Delta t$ because we are already looking at rates.

To get the ERA5 quarterly rate of height change due to SMB, we can either:

1. Calculate the rate of change for each month, and average those rates quarterly; or,
2. Calculate the rate of change between on-quarter time steps and discard other time steps.

I prefer #1 - mathematically and to make it more adaptable to other groupings.

**The comparison to ATL15 is coming out weird, but if we normalize ERA5 to a single long-term climatology instead of monthly values then it looks good. :shrug:**

### Convert units and normalize

In [ ]:
# Convert ERA5 SMB to m/a ice equivalent
# -- Gt/m2 * 1E12 = mm/mo water equivalent
# -- mm/mo w.e. * 1E-3 [m/mm] * 12 [mo/a] * (rho_water / rho_ice) [i.e./w.e.] = m/a ice equivalent
# conversion_factor = 1E9 * 12 * (1 / 0.917) # Gt/m2 to m/a ice equivalent
# era5_units = r"$[m a^{-1} ice equiv.]$"
conversion_factor = 1E9 * 12 # Gt/m2 to m/a water equivalent
era5_units = r"$[m\ a^{-1}\ water\ equiv.]$"
era5['SMB_anom_abs'] *= conversion_factor
era5['clim_mean'] *= conversion_factor
era5['clim_std'] *= conversion_factor

In [ ]:
# Normalize ERA5 SMB to a long-term mean SMB instead of monthly means
era5_prenorm = era5.copy()
era5 = xr.full_like(era5_prenorm.clim_mean, fill_value=np.nan).to_dataset()
era5['clim_mean'] = era5_prenorm['clim_mean'].mean(dim='time')
era5['SMB_anom_abs'] = (era5_prenorm.SMB_anom_abs + era5_prenorm.clim_mean) - era5.clim_mean
era5

In [ ]:
# Calculate rate of height change due to SMB
# -- dSMBAdt = (SMB_A + SMB_mean)_t1 - (SMB_A + SMB_mean)_t0
# -- NOTE: time labels are for right edge (later time) of each rate
era5['dSMBAdt'] = era5.SMB_anom_abs.diff(dim='time')
era5

In [ ]:
# Plot check
t_i = 5
fig, ax = plt.subplots()
ax = is2smb.viz.add_subplot_southpolarstereo(fig, ax, grid=True)
props = props_var['dsmbadt'].copy()

era5.dSMBAdt.isel(time=t_i).plot(
    ax=ax,
    cmap=props['cmap'],
    norm=props['cnorm'],
    cbar_kwargs=dict(
        label=f"{props['eqn_label']}"
    )
)
ax.set_title(f"{props['name']}\n{era5.time.values[t_i-1].item().date()} through {era5.time.values[t_i].item().date()}")
plt.show()

### Aggregate to quarterly

In [ ]:
# Calculate quarterly SMB anomaly and rate of height change due to SMB

def aggregate_SMBA_quarterly(data, i_centers, dt):
    # output times are for *center* of quarter
    i_centers = np.array(i_centers)
    i = i_centers[0]
    smb_anom_abs = data.isel(time=slice(i-1, i+2)).sum(dim='time', min_count=1)
    smb_anom_abs = smb_anom_abs.assign_coords(time=data.isel(time=i-1).time.item())
    # dsmbadt = (data.isel(time=i+1) - data.isel(time=i-1)) / dt
    # dsmbadt = dsmbadt.assign_coords(time=data.isel(time=i-1).time.item())
    for i in i_centers[1:]:
        next_smb = data.isel(time=slice(i-1, i+2)).sum(dim='time', min_count=1)
        next_smb = next_smb.assign_coords(time=data.isel(time=i-1).time.item())
        smb_anom_abs = xr.concat([smb_anom_abs, next_smb], dim='time')
    # dsmbadt = how much SMB anomaly changed from i-1 to i+1, over time
    dsmbadt = (smb_anom_abs - data.isel(time=i_centers-1))# / dt
    dsmbadt = dsmbadt.rename('dSMBAdt')
    return smb_anom_abs, dsmbadt

# futzing with dates and types to match model to ATL15
dt = 0.25
quarters = [
    f'{SOTC_YEAR}-01-01', 
    f'{SOTC_YEAR}-04-01', 
    f'{SOTC_YEAR}-07-01', 
    f'{SOTC_YEAR}-10-01', 
    f'{SOTC_YEAR+1}-01-01'
]
quarters = [np.datetime64(x) for x in quarters]
decimal_dates = np.arange(SOTC_YEAR+(dt/2), SOTC_YEAR+1+(dt/2), step=dt)

# calculate total SMB anomaly, and rate of change of SMB
smb_anom_abs, dsmbadt = aggregate_SMBA_quarterly(era5.SMB_anom_abs, i_centers=[1,4,7,10], dt=0.25)
era5_quarterly = xr.Dataset(data_vars=dict(
    SMB_anom_abs=smb_anom_abs,
    dSMBAdt=dsmbadt,
))
era5_quarterly = era5_quarterly.assign_coords(coords=dict(time=decimal_dates))
era5_quarterly = era5_quarterly.assign_attrs(dict(name="Quarterly ERA5"))

In [ ]:
# plot check:
dt = 0.25
fig, ax = plt.subplots(
    nrows=1,
    ncols=4,
    figsize=(10,3.5),
    layout='compressed',
)
ax = is2smb.viz.add_subplot_southpolarstereo(fig, ax, grid=True)
props = props_var['dsmbadt'].copy()
# props['cnorm'].halfrange *= 3

for i, a in enumerate(ax.flat):
    h = era5_quarterly.dSMBAdt.isel(time=i).plot(
        ax=a,
        cmap=props['cmap'],
        norm=props['cnorm'],
        add_colorbar=False, 
    )
    time_range = [
        era5_quarterly.time.values[i] - dt/2,
        era5_quarterly.time.values[i] + dt/2
    ]
    a.set_title(f"{time_range[0]} through {time_range[1]}", fontsize=10)

fig.colorbar(h, ax=ax[-1], extend='both', label=props['eqn_label'])
fig.suptitle(f"{era5_quarterly.attrs['name']}: {props['name']}", fontweight='bold')
plt.show()

## Detrend data

Here we are working with the rate of height change (dhdt) and the rate of height change due to SMB (dSMBAdt, where SMBA is the cumulative SMB anomaly). These are both rates ($[m\ a^{-1}]$), so detrending is simply removing the mean (y=constant) rate of height change.

(note: probably don't need to actually do this, at least for the dSMBAdt because SMB_A is already relative to the mean, but do it anyway to maintain equivalence with dhdt)

In [ ]:
# Detrend quarterly and annual dhdt (remove the mean) for ATL15
for L in ['dhdt_lag1', 'dhdt_lag4']:
    atl15[L]['dhdt'] = is2smb.spatial.detrend_da(
        da=atl15[L].dhdt,
        dim='time',
        degree=0
    )
    atl15[L]['dhdt_trend'] = atl15_original[L]['dhdt'] - atl15[L]['dhdt']

In [ ]:
# Detrend monthly and quarterly dSMBAdt (remove the mean) for ERA5
for src in [era5, era5_quarterly]:
    pre_detrend = src.dSMBAdt.copy()
    src['dSMBAdt'] = is2smb.spatial.detrend_da(
        da=src.dSMBAdt,
        dim='time',
        degree=0
    )
    src['dSMBAdt_trend'] = pre_detrend - src['dSMBAdt']

# ATL15

## Unmodified heights

### Height anomaly (`delta_h`)

In [ ]:
data = atl15.delta_h
times = data.time.values
props = props_var['dh'].copy()

fig, ax = plt.subplots(
    nrows=1,
    ncols=len(times),
    figsize=(10,4),
    layout='compressed',
)
ax = is2smb.viz.add_subplot_southpolarstereo(fig, ax, grid=True)

for i, t in enumerate(times):
    h = data.delta_h.sel(time=t).plot(
        ax=ax[i],
        cmap=props['cmap'],
        norm=props['cnorm'],
        add_colorbar=False,
    )
    ax[i].set_title(t)

fig.colorbar(h, ax=ax[-1], extend='both', label=props['short_name'])
fig.suptitle(
    f"""\
    Quarterly height relative to datum surface (1 January 2020)
    ICESat-2 ATL15, {atl15_spatial_resolution}
    """,
    fontweight='bold'
)

plt.show()

### Quarterly height change rate (`dhdt_lag1`)

In [ ]:
# Plot: maps

dt = 0.25
data = atl15.dhdt_lag1
times = data.time.values
props = props_var['dhdt'].copy()

fig, ax = plt.subplots(
    nrows=1,
    ncols=len(times),
    figsize=(10,4),
    layout='compressed',
)
ax = is2smb.viz.add_subplot_southpolarstereo(fig, ax, grid=True)

for i, t in enumerate(times):
    h = data.dhdt.sel(time=t).plot(
        ax=ax[i],
        cmap=props['cmap'],
        norm=props['cnorm'],
        add_colorbar=False,
    )
    ttxt = f'{t - dt/2} to {t + dt/2}'
    ax[i].set_title(ttxt)

fig.colorbar(h, ax=ax[-1], extend='both', label=f"{props['short_name']} {props['units']}")
fig.suptitle(
    f"""\
    Quarterly rate of height change
    ICESat-2 ATL15, {atl15_spatial_resolution}
    """,
    fontweight='bold'
)

plt.show()

In [ ]:
# Plot: time series of integrated volume change
vol_km3 = (data.dhdt * data.ice_area) / 1E9
integrated_volume_rate = vol_km3.sum(
    skipna=True,
    min_count=1,
    dim=['x', 'y']
)

fig, ax = plt.subplots(
    figsize=(5,2),
)
integrated_volume_rate.plot(
    ax=ax, 
    marker='.'
)
ax.set_xlabel('time')
ax.set_ylabel(r'volume flux [$km^3\ a^{-1}$]')
ax.set_title('Integrated quarterly volume flux')
plt.show()

### Annual height change rate (`dhdt_lag4`)

In [ ]:
dt = 1
data = atl15.dhdt_lag4
times = data.time.values
props = props_var['dhdt'].copy()

fig, ax = plt.subplots(
    nrows=1,
    ncols=len(times),
    figsize=(10,4),
    layout='compressed',
)
ax = is2smb.viz.add_subplot_southpolarstereo(fig, ax, grid=True)


for i, t in enumerate(times):
    h = data.dhdt.sel(time=t).plot(
        ax=ax[i],
        cmap=props['cmap'],
        norm=props['cnorm'],
        add_colorbar=False,
    )
    # ax[i].set_frame_on(False)
    ttxt = f'{t - dt/2} to {t + dt/2}'
    ax[i].set_title(ttxt)

fig.colorbar(h, ax=ax[-1], extend='both', label=f"{props['short_name']} {props['units']}")
fig.suptitle(
    f"""\
    Annual rate of height change
    ICESat-2 ATL15, {atl15_spatial_resolution}
    """,
    fontweight='bold'
)

plt.show()

In [ ]:
# Plot: time series of integrated volume change
vol_km3 = (data.dhdt * data.ice_area) / 1E9
integrated_volume_rate = vol_km3.sum(
    skipna=True,
    min_count=1,
    dim=['x', 'y']
)

fig, ax = plt.subplots(
    figsize=(5,2),
)
integrated_volume_rate.plot(
    ax=ax, 
    marker='.'
)
ax.set_xlabel('time')
ax.set_ylabel(r'volume flux [$km^3\ a^{-1}$]')
ax.set_title('Integrated annual volume flux')
plt.show()

## Detrended heights

### Quarterly height change rate (`dhdt_lag1`)

In [ ]:
# # Plot: trend and quarterly maps

# dt = 0.25
# lag = 'dhdt_lag1'

# data = atl15[lag].sel(time=slice(SOTC_YEAR - dt/2, None))
# times = data.time.values
# props = props_var['dhdt'].copy()

# fig, ax = plt.subplots(
#     nrows=1,
#     ncols=len(times)+1,
#     figsize=(12,3.25),
#     layout='compressed',
# )
# ax = is2smb.viz.add_subplot_southpolarstereo(fig, ax, grid=True)

# # plot trend
# atl15[lag].dhdt_trend.isel(time=-1).plot(
#     ax=ax[0],
#     cmap=props['cmap'],
#     norm=props['cnorm'],
#     cbar_kwargs=dict(
#         extend='both',
#         label=f"{props['short_name']} {props['units']}",
#     ),
# )
# ttxt = f'Trend, {atl15[lag].time.values[0] - dt/2} to {atl15[lag].time.values[-1] + dt/2}'
# ax[0].set_title(ttxt)

# for i, t in enumerate(times):
#     h = data.dhdt.sel(time=t).plot(
#         ax=ax[i+1],
#         cmap=props['cmap'],
#         norm=props['cnorm'],
#         add_colorbar=False,
#     )
#     ttxt = f'dhdt, {t - dt/2} to {t + dt/2}'
#     ax[i+1].set_title(ttxt)

# # prettify
# fig.colorbar(h, ax=ax[-1], extend='both', label=f"{props['short_name']} {props['units']}")
# fig.suptitle(
#     f"""\
#     Quarterly rate of height change (detrended)
#     ICESat-2 ATL15, {atl15_spatial_resolution}
#     """,
#     fontweight='bold'
# )

# plt.show()

### Annual height change rate (`dhdt_lag4`)

In [ ]:
# # Plot: trend and annual maps

# dt = 1
# lag = 'dhdt_lag4'

# data = atl15[lag]
# times = data.time.values
# props = props_var['dhdt'].copy()

# fig, ax = plt.subplots(
#     nrows=1,
#     ncols=len(times)+1,
#     figsize=(12,3.5),
#     layout='compressed',
# )
# ax = is2smb.viz.add_subplot_southpolarstereo(fig, ax, grid=True)

# # plot trend
# atl15[lag].dhdt_trend.isel(time=-1).plot(
#     ax=ax[0],
#     cmap=props['cmap'],
#     norm=props['cnorm'],
#     cbar_kwargs=dict(
#         extend='both',
#         label=f"{props['short_name']} {props['units']}",
#     ),
# )
# ttxt = f'Trend, {atl15[lag].time.values[0] - dt/2} to {atl15[lag].time.values[-1] + dt/2}'
# ax[0].set_title(ttxt)

# for i, t in enumerate(times):
#     h = data.dhdt.sel(time=t).plot(
#         ax=ax[i+1],
#         cmap=props['cmap'],
#         norm=props['cnorm'],
#         add_colorbar=False,
#     )
#     ttxt = f'dhdt, {t - dt/2} to {t + dt/2}'
#     ax[i+1].set_title(ttxt)

# # prettify
# fig.colorbar(h, ax=ax[-1], extend='both', label=f"{props['short_name']} {props['units']}")
# fig.suptitle(
#     f"""\
#     Annual rate of height change (detrended)
#     ICESat-2 ATL15, {atl15_spatial_resolution}
#     """,
#     fontweight='bold'
# )

# plt.show()

# ERA5

In [ ]:
# Absolute SMB anomaly, quarterly
dt = 0.25
fig, ax = plt.subplots(
    nrows=1,
    ncols=4,
    figsize=(10,3.5),
    layout='compressed',
)
ax = is2smb.viz.add_subplot_southpolarstereo(fig, ax, grid=True)
props = props_var['smba'].copy()
props['cnorm'].halfrange *= 3

for i, a in enumerate(ax.flat):
    h = era5_quarterly.SMB_anom_abs.isel(time=i).plot(
        ax=a,
        cmap=props['cmap'],
        norm=props['cnorm'],
        add_colorbar=False, 
    )
    time_range = [
        era5_quarterly.time.values[i] - dt/2,
        era5_quarterly.time.values[i] + dt/2
    ]
    a.set_title(f"{time_range[0]} through {time_range[1]}", fontsize=10)

fig.colorbar(h, ax=ax[-1], extend='both', label=props['eqn_label'])
fig.suptitle(f"{era5_quarterly.attrs['name']}: {props['name']}", fontweight='bold')
plt.show()

In [ ]:
# # Percent SMB anomaly, quarterly
# dt = 0.25
# fig, ax = plt.subplots(
#     nrows=1,
#     ncols=4,
#     figsize=(10,3.5),
#     layout='compressed',
# )
# ax = is2smb.viz.add_subplot_southpolarstereo(fig, ax, grid=True)
# props = dict(
#     cmap=mcmap['BrBG'],
#     cnorm=colors.CenteredNorm(vcenter=0, halfrange=500),
#     name="surface mass balance anomaly (%)",
#     eqn_label="SMB anomaly (%)",
# )

# for i, a in enumerate(ax.flat):
#     h = era5_quarterly.SMB_anom_pct.isel(time=i).plot(
#         ax=a,
#         cmap=props['cmap'],
#         norm=props['cnorm'],
#         add_colorbar=False, 
#     )
#     time_range = [
#         era5_quarterly.time.values[i] - dt/2,
#         era5_quarterly.time.values[i] + dt/2
#     ]
#     a.set_title(f"{time_range[0]} through {time_range[1]}", fontsize=10)

# fig.colorbar(h, ax=ax[-1], extend='both', label=props['eqn_label'])
# fig.suptitle(f"{era5_quarterly.attrs['name']}: {props['name']}", fontweight='bold')
# plt.show()

In [ ]:
# Rate of height change from SMB, quarterly
dt = 0.25
fig, ax = plt.subplots(
    nrows=1,
    ncols=4,
    figsize=(10,3.5),
    layout='compressed',
)
ax = is2smb.viz.add_subplot_southpolarstereo(fig, ax, grid=True)
props = props_var['dsmbadt'].copy()

for i, a in enumerate(ax.flat):
    h = era5_quarterly.dSMBAdt.isel(time=i).plot(
        ax=a,
        cmap=props['cmap'],
        norm=props['cnorm'],
        add_colorbar=False, 
    )
    time_range = [
        era5_quarterly.time.values[i] - dt/2,
        era5_quarterly.time.values[i] + dt/2
    ]
    a.set_title(f"{time_range[0]} through {time_range[1]}", fontsize=10)

fig.colorbar(h, ax=ax[-1], extend='both', label=props['eqn_label'])
fig.suptitle(f"{era5_quarterly.attrs['name']}: {props['name']}", fontweight='bold')
plt.show()

# Compare ATL15 and ERA5

The most apples-to-apples comparison is the ATL15 dhdt versus the ERA5 rate of height change due to SMB. (It's more of a Granny Smith to Cosmic Crisp comparison)

In [ ]:
# Quarterly rate of height change for ATL15 and ERA5
dt = 0.25
times = era5_quarterly.dSMBAdt.time.values
fig, ax = plt.subplots(
    nrows=2,
    ncols=4,
    figsize=(10,7),
    layout='compressed'
)
ax = is2smb.viz.add_subplot_southpolarstereo(fig, ax, grid=True)
props = dict(
    # cmap=props_var['dsmbadt']['cmap'].copy(),
    cmap=mcmap['RdBu'],
    norm=colors.CenteredNorm(vcenter=0, halfrange=0.5),
    add_colorbar=False,
)

for i in np.arange(4):
    t = times[i]
    # top row: ERA5
    h_era5 = era5_quarterly.dSMBAdt.sel(time=t).plot(
    # h_era5 = era5_quarterly.SMB_anom_abs.sel(time=t).plot(
        ax=ax[0,i],
        **props
    )
    ax[0,i].set_title(f"{t - dt/2} through {t + dt/2}")
    # bottom row: ATL15
    if t in atl15.dhdt_lag1.time.values:
        h_atl15 = atl15.dhdt_lag1.dhdt.sel(time=t).plot(
            ax=ax[1,i],
            **props
        )
    else: ax[1,i].set_visible(False)
    ax[1,i].set_title(None)
# ax[0,0].set_ylabel("ERA5")
# ax[1,0].set_ylabel("ATL15")
[a.set_frame_on(False) for a in ax.flat]
fig.colorbar(h_era5, ax=ax[0,-1], extend='both', label=props_var['dsmbadt']['eqn_label'])
fig.colorbar(h_atl15, ax=ax[1,-1], extend='both', label=props_var['dhdt']['eqn_label'])
# fig.suptitle()
plt.show()

In [ ]:
temp = era5_quarterly.dSMBAdt - atl15.dhdt_lag1.dhdt
numer = era5_quarterly.dSMBAdt.sel(time=temp.time.values[0])
denom = atl15.dhdt_lag1.dhdt.sel(time=temp.time.values[0])

props = is2smb.viz.cprops_relative_difference()

fig, ax = plt.subplots()
np.abs(numer/denom).plot(
    ax=ax,
    cmap=props['cmap'],
    norm=props['cnorm'],
    cbar_kwargs=dict(
        label=f"{numer.name} \N{DIVISION SIGN} {denom.name} $[unitless]$",
        ticks=props['cticks'],
    ),
)
# plot areas where sign is different (relative dSMBAdt is negative)
ax.contourf(
    numer.x,
    numer.y,
    (numer/denom)<0,
    hatches=[None,'xxx'],
    alpha=0,
)
print('HASHED AREAS IN RELATIVE DIFFERENCE ARE NEGATIVE')
txt = f"""\
    Relative difference in {props_var['dsmbadt']['eqn_name']}
    ({numer.name}\N{DIVISION SIGN}{denom.name})\
"""

In [ ]:
# temp = era5_quarterly.SMB_anom_abs - atl15.dhdt_lag1.dhdt
# x = era5_quarterly.SMB_anom_abs.sel(time=temp.time)
temp = era5_quarterly.dSMBAdt - atl15.dhdt_lag1.dhdt
x = atl15.dhdt_lag1.dhdt.sel(time=temp.time)
y = era5_quarterly.dSMBAdt.sel(time=temp.time)

fit = is2smb.statistics.fit_linear(x, y)

fig, ax = plt.subplots()
cmap = mcmap['plasma']
# cmap.set_bad('silver')
norm = colors.Normalize(vmax=20)
cmin = 1
h2d = is2smb.viz.plot_hist2d(
    ax=ax,
    x=x,
    y=y,
    cmap=cmap,
    cmin=cmin,
    norm=norm,
)
l = is2smb.viz.plot_fit(
    ax=ax,
    fit=fit['fit'],
)
ax.set_xlabel("ATL15 dhdt")
ax.set_ylabel("ERA5 dSMBAdt")
ax.grid()
ax.set_aspect('equal')
ax.legend(loc='lower right')
plt.show()

# Final (?) fig

In [ ]:
# Quarterly rate of height change for ATL15 and ERA5
dt = 0.25
times = (era5_quarterly.dSMBAdt - atl15.dhdt_lag1.dhdt).time.values

fig, ax = plt.subplots(
    nrows=2,
    ncols=2,
    figsize=(8,6),
    layout='compressed'
)
ax = is2smb.viz.add_subplot_southpolarstereo(fig, ax, grid=True)
props = dict(
    cmap=mcmap['RdBu'],
    norm=colors.CenteredNorm(vcenter=0, halfrange=0.5),
    add_colorbar=False,
)

for i in [0,1]:
    t = times[i]
    # top row: ERA5
    h_era5 = era5_quarterly.dSMBAdt.sel(time=t).plot(
    # h_era5 = era5_quarterly.SMB_anom_abs.sel(time=t).plot(
        ax=ax[0,i],
        **props
    )
    # ax[0,i].set_title(f"{t - dt/2} through {t + dt/2}")
    ax[0,i].set_ylabel("ERA5")
    # bottom row: ATL15
    h_atl15 = atl15.dhdt_lag1.dhdt.sel(time=t).plot(
        ax=ax[1,i],
        **props
    )
    # ax[1,i].set_title(None)
[a.set_title(None) for a in ax.flat]
[a.set_frame_on(False) for a in ax.flat]

def ax_title(ax, r, c, mos, src):
    s = f"{src}\n{times[c]-dt/2}-{times[c]+dt/2} {mos}"
    ax[r,c].text(x=0.05, y=0.95, s=s, transform=ax[r,c].transAxes)
ax_title(ax, 0, 0, mos="(Jan-Mar)", src="ERA5 rate of height change from SMB")
ax_title(ax, 0, 1, mos="(Apr-Jun)", src='')
ax_title(ax, 1, 0, mos="(Jan-Mar)", src="ICESat-2 rate of surface height change")
ax_title(ax, 1, 1, mos="(Apr-Jun)", src='')

fig.colorbar(h_era5, ax=ax[0,-1], extend='both', label=r"$dhdt_{SMB}$"+f" {era5_units}")
fig.colorbar(h_atl15, ax=ax[1,-1], extend='both', label=r"$dhdt_{total}$"+f" {props_var['dhdt']['units']}")
# fig.suptitle()
# fig.savefig("era5_vs_is2_maps", format='svg', transparent=True, dpi=300)
fig.savefig("era5_vs_is2_maps.png", format='png', transparent=True, dpi=300)
fig.savefig("era5_vs_is2_maps.pdf", format='pdf', transparent=True, dpi=300)
plt.show()

In [ ]:
# 2d histogram
temp = era5_quarterly.dSMBAdt - atl15.dhdt_lag1.dhdt
x = atl15.dhdt_lag1.dhdt.sel(time=temp.time)
y = era5_quarterly.dSMBAdt.sel(time=temp.time)

fit = is2smb.statistics.fit_linear(x, y)

fig, ax = plt.subplots(figsize=(7,4))
cmap = mcmap['plasma']
# cmap.set_bad('silver')
norm = colors.LogNorm(vmax=100)
cmin = 1
h2d = is2smb.viz.plot_hist2d(
    ax=ax,
    x=x,
    y=y,
    cmap=cmap,
    cmin=cmin,
    norm=norm,
)
l = is2smb.viz.plot_fit(
    ax=ax,
    fit=fit['fit'],
)
ax.set_xlabel(f"ICESat-2 rate of surface height change {props_var['dhdt']['units']}")
ax.set_ylabel(f"ERA5 rate of height change from SMB\n{era5_units}")
ax.grid()
# ax.set_aspect('equal')
ax.legend(loc='lower right')
fig.colorbar(h2d[-1], ax=ax, extend='max', label='count')
fig.savefig("era5_vs_is2_hist.png", format='png', transparent=True, dpi=300)
fig.savefig("era5_vs_is2_hist.pdf", format='pdf', transparent=True, dpi=300)
plt.show()